In [1]:
import os
print(os.getcwd())

/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity


In [20]:
import pandas as pd
import numpy as np
import itertools
import os

def calcula_mse(conjunto):

    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    # Calcula o MSE corretamente
    mse = np.mean([(Z - z_pred) ** 2 for z_pred in z_preds])

    print(f"mse: {mse}")
    return mse

def diversity(conjunto):

    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    
    Z_pred_total = np.hstack(z_preds)
   # Z_media = valores.mean()
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    # Cálculo do erro:
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    #print (f'error1: {error1}')
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    #print (f'error2: {error2}')
    error = error1 + error2
    print (f'error: {error}')
    return error.T



def filtrar_combinacoes(file_list, tamanho_conjunto, erro_max):
    """Gera e filtra combinações de arquivos baseado no MSE e erro específico."""
    combinacoes = itertools.combinations(file_list, tamanho_conjunto)

    # Lista para armazenar combinações filtradas
    resultados = []

    # Filtra apenas as combinações que passam nos critérios
    for conjunto in combinacoes:
        mse = calcula_mse(conjunto)
        erro = diversity(conjunto)
        if np.all(erro < erro_max):  # Passa no filtro do erro
            resultados.append({"Conjunto": conjunto, "MSE": mse, "Erro de Diversidade": erro})
              
    return resultados  # Retorna os conjuntos válidos

def salvar_resultados(resultados, nome_arquivo):
    """Salva os resultados em uma única planilha."""
    df_resultado = pd.DataFrame(resultados)
    
    # Salvando o DataFrame em um arquivo Excel
    df_resultado.to_excel(nome_arquivo, index=False)
    print(f'Arquivo {nome_arquivo} salvo com sucesso!')

# Lista de arquivos disponíveis no diretório atual
file_list = [f for f in os.listdir() if f.endswith(".xlsx")]

# Parâmetros
tamanho_conjunto = 10
erro_max = 1000  # Defina um valor para o erro aceitável

# Executa o filtro e obtém os conjuntos válidos
resultados_validos = filtrar_combinacoes(file_list, tamanho_conjunto, erro_max)

# Lista de arquivos disponíveis no diretório atual
file_list = [f for f in os.listdir() if f.endswith(".xlsx")]

# Salva os resultados em uma única planilha
salvar_resultados(resultados_validos, "resultados.xlsx")




mse: 44082.07829401206
error: [[5579372.18465362]
 [5397053.2831256 ]
 [5124570.18518097]
 ...
 [1982262.0850626 ]
 [2180590.65914406]
 [2378039.65667632]]
mse: 44088.3709773979
error: [[5258513.13469329]
 [5159854.17469822]
 [4961093.78575265]
 ...
 [1999467.72301283]
 [2170807.03001592]
 [2329274.55513591]]
mse: 44039.8040967073


KeyboardInterrupt: 

In [ ]:
#eu quero ler todos os minhas plnailhas e depois conseguir tirar a media de todos os valores

import pandas as pd
import numpy as np
import itertools
import os
from tqdm import tqdm  # Para barra de progresso

def diversity( file_paths):
    """
    Calcula o erro para um conjunto de redes fornecido e salva os resultados em um arquivo Excel.
    """
    dataframes = [pd.read_excel(path) for path in file_paths]
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in dataframes]
    Z_pred_total = np.hstack(z_preds)
   # Z_media = valores.mean()
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    #print(f'Z_med: {Z_media}')

    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    # Cálculo do erro
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    #print (f'error1: {error1}')
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    #print (f'error2: {error2}')
    error = error1 + error2
    #print (f'error: {error}')
    return error

def mse(file_paths):
    dataframes = [pd.read_excel(path) for path in file_paths]
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in dataframes]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    
    # Calcula o erro quadrático médio para cada Z_pred
    mse_values = [(Z - z_pred) ** 2 for z_pred in z_preds]
    
    # Concatena todas as diferenças ao quadrado e calcula a média
    mse = np.mean(np.hstack(mse_values))
    
    print(f'MSE: {mse}')
    return mse

# Lista dos arquivos das 100 redes (substitua pelos nomes reais dos arquivos)
file_list = ["1000_model_18_4_13.xlsx", "1000_model_24_9_15.xlsx", "1000_model_24_9_13.xlsx",
    "1000_model_18_4_1.xlsx", "1000_model_14_9_9.xlsx", "1000_model_14_9_22.xlsx",
    "1000_model_18_4_20.xlsx", "1000_model_37_9_20.xlsx", "1000_model_9_9_4.xlsx",
    "1000_model_9_9_0.xlsx"]

diversity(file_list)
mse(file_list)


Z_med: [[3334.06376953]
 [3188.70734863]
 [3039.83764648]
 ...
 [1395.08121338]
 [1494.13549805]
 [1592.76551514]]
error1: [[5528158.00719082]
 [5368071.13184757]
 [5108709.7342431 ]
 ...
 [1877940.90522739]
 [2024486.48263623]
 [2151595.71470411]]
error2: [[ 51214.17746279]
 [ 28982.15127802]
 [ 15860.45093787]
 ...
 [104321.1798352 ]
 [156104.17650783]
 [226443.94197221]]
error: [[5579372.18465362]
 [5397053.2831256 ]
 [5124570.18518097]
 ...
 [1982262.0850626 ]
 [2180590.65914406]
 [2378039.65667632]]
MSE: 44082.07829401207


44082.07829401207

In [24]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
from scipy.spatial import Delaunay


def diversity(fileName, file_path_1,file_path_2,file_path_3,file_path_4,file_path_5,file_path_6,file_path_7, file_path_8,file_path_9,file_path_10):

  
    df_1=pd.read_excel(file_path_1)
    df_2=pd.read_excel(file_path_2)
    df_3=pd.read_excel(file_path_3)
    df_4 = pd.read_excel(file_path_4)
    df_5 = pd.read_excel(file_path_5)
    df_6 = pd.read_excel(file_path_6)
    df_7= pd.read_excel(file_path_7)
    df_8 = pd.read_excel(file_path_8)
    df_9 = pd.read_excel(file_path_9)
    df_10= pd.read_excel(file_path_10)
   
  
    z_pred_1 = df_1['Z_pred'].values
    z_pred_2 = df_2['Z_pred'].values
    z_pred_3 = df_3['Z_pred'].values
    z_pred_4 = df_4['Z_pred'].values
    z_pred_5 = df_5['Z_pred'].values
    z_pred_6 = df_6['Z_pred'].values
    z_pred_7 = df_7['Z_pred'].values
    z_pred_8 = df_8['Z_pred'].values
    z_pred_9 = df_9['Z_pred'].values
    z_pred_10 = df_10['Z_pred'].values
    
    
    Z_pred_1 = z_pred_1.reshape(-1, 1)
    Z_pred_2 = z_pred_2.reshape(-1, 1)
    Z_pred_3 = z_pred_3.reshape(-1, 1)
    Z_pred_4 = z_pred_4.reshape(-1, 1)
    Z_pred_5 = z_pred_5.reshape(-1, 1)
    Z_pred_6 = z_pred_6.reshape(-1, 1)
    Z_pred_7 = z_pred_7.reshape(-1, 1)
    Z_pred_8 = z_pred_8.reshape(-1, 1)
    Z_pred_9 = z_pred_9.reshape(-1, 1)
    Z_pred_10 = z_pred_10.reshape(-1, 1)


    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    

    Z_pred_total = np.hstack([Z_pred_1, Z_pred_2, Z_pred_3, Z_pred_4, Z_pred_5,
                           Z_pred_6, Z_pred_7, Z_pred_8, Z_pred_9, Z_pred_10])

    Z_pred = np.mean(Z_pred_total, axis=1, keepdims=True)
    print(f'Z_med: {Z_pred}')
    
    error1= (Z-Z_pred_1)**2 +(Z-Z_pred_2)**2+(Z-Z_pred_3)**2+(Z-Z_pred_4)**2+(Z-Z_pred_5)**2+(Z-Z_pred_6)**2+(Z-Z_pred_7)**2+(Z-Z_pred_8)**2+(Z-Z_pred_9)**2+(Z-Z_pred_10)**2
    error2= (Z_pred_1-Z_pred)**2 +(Z_pred_2-Z_pred)**2+(Z_pred_3-Z_pred)**2+(Z_pred_4-Z_pred)**2+(Z_pred_5-Z_pred)**2+(Z_pred_6-Z_pred)**2+(Z_pred_7-Z_pred)**2+(Z_pred_8-Z_pred)**2+(Z_pred_9-Z_pred)**2+(Z_pred_10-Z_pred)**2
    print (error2)
    error = error1+error2
    
    print(f'Error shape: {error.shape}')  # Verificando formato

    # Criando DataFrame para salvar no Excel
    df_error = pd.DataFrame({'Error': error.flatten()})

    # Salvando a planilha
    df_error.to_excel(f"{fileName}.xlsx", index=False)
    print(f'Arquivo {fileName}.xlsx salvo com sucesso!')

    


diversity= diversity("1000_model_error.xlsx","1000_model_18_4_13.xlsx", "1000_model_24_9_15.xlsx", "1000_model_24_9_13.xlsx",
    "1000_model_18_4_1.xlsx", "1000_model_14_9_9.xlsx", "1000_model_14_9_22.xlsx",
    "1000_model_18_4_20.xlsx", "1000_model_37_9_20.xlsx", "1000_model_9_9_4.xlsx",
    "1000_model_9_9_0.xlsx")

#fig = plot_figure("1000_model_error.xlsx","1000_model_med.xlsx")



Z_med: [[3334.06376953]
 [3188.70734863]
 [3039.83764648]
 ...
 [1395.08121338]
 [1494.13549805]
 [1592.76551514]]
[[ 51214.17746279]
 [ 28982.15127802]
 [ 15860.45093787]
 ...
 [104321.1798352 ]
 [156104.17650783]
 [226443.94197221]]
Error shape: (1024, 1)
Arquivo 1000_model_error.xlsx.xlsx salvo com sucesso!
